In [1]:
import os
current_dir = os.getcwd()
parent_dir = os.path.abspath(os.path.join(current_dir,os.pardir))
os.chdir(parent_dir)

In [2]:
import numpy as np
import pandas as pd
import torch
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import seaborn as sns
from vae_earlystopping import EarlyStopping
from model1.m_std_05 import Model3
from loss1.l2 import loss2


In [3]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [4]:
train_loader = torch.load('torch_1/pre_retrain_loader_min.pt',weights_only=False)
val_loader = torch.load('torch_1/pre_reval_loader_min.pt',weights_only=False)
test_loader = torch.load('torch_1/pre_retest_loader_min.pt',weights_only=False)


In [5]:
x_sample,x2_sample, c_sample = next(iter(train_loader))
x_dim = x_sample.shape[1]
x2_dim = x2_sample.shape[1]
c_dim = c_sample.shape[1]
x_dim, x2_dim, c_dim

(23, 23, 15)

In [6]:
model = Model3(x_dim,x2_dim, c_dim,z_dim=8,z2_dim = 8).to(device)
early_stopping = EarlyStopping(patience=40,min_delta = 1e-9)
optimizer = optim.Adam(model.parameters(),lr = 1e-3, weight_decay=1e-5)
epochs = 600

In [7]:
for epoch in range(1,epochs+1):
    model.train()
    t_loss = 0
    for x, x2, c in train_loader:
        x,x2,c = x.to(device),x2.to(device),c.to(device)
        optimizer.zero_grad()
        bce_logit, x_hat,mu,logvar,mu2,logvar2 = model(x,x2,c)
        loss_dict = loss2(bce_logit,x_hat,x,x2,mu,logvar,mu2,logvar2)
        loss_dict['loss'].backward()
        optimizer.step()
        t_loss +=loss_dict['loss'].item()
    

    model.eval()
    v_loss = 0
    with torch.no_grad():
        for v_x, v2_x, v_c in val_loader:
            v_x,v2_x, v_c = v_x.to(device),v2_x.to(device), v_c.to(device)
            v_bce_logit,v_x_hat,v_mu,v_logvar,v2_mu,v2_logvar = model(v_x, v2_x, v_c)
            loss_dict = loss2(v_bce_logit, v_x_hat, v_x,v2_x, v_mu,v_logvar,v2_mu,v2_logvar,a=1,b=1,c=1,d=1)
            v_loss +=loss_dict['loss'].item()
    avg_train_loss = t_loss/len(train_loader)
    avg_val_loss = v_loss/len(val_loader)

    if epoch % 20 ==0  or epoch ==2:
        print(f'Epoch [{epoch}/{epochs}]|Train:{avg_train_loss:.4f} |Val:{avg_val_loss:.4f}')
    if early_stopping(avg_val_loss,model):
        break


Epoch [2/600]|Train:3.6169 |Val:3.5030
EarlyStopping counter: 1 out of 40
Epoch [20/600]|Train:2.2004 |Val:2.3512
EarlyStopping counter: 1 out of 40
EarlyStopping counter: 1 out of 40
EarlyStopping counter: 2 out of 40
EarlyStopping counter: 1 out of 40
EarlyStopping counter: 1 out of 40
EarlyStopping counter: 1 out of 40
EarlyStopping counter: 2 out of 40
Epoch [40/600]|Train:1.6763 |Val:1.8189
EarlyStopping counter: 1 out of 40
EarlyStopping counter: 2 out of 40
EarlyStopping counter: 3 out of 40
EarlyStopping counter: 1 out of 40
EarlyStopping counter: 1 out of 40
EarlyStopping counter: 2 out of 40
EarlyStopping counter: 3 out of 40
EarlyStopping counter: 1 out of 40
EarlyStopping counter: 2 out of 40
EarlyStopping counter: 3 out of 40
EarlyStopping counter: 1 out of 40
EarlyStopping counter: 2 out of 40
Epoch [60/600]|Train:1.4312 |Val:1.6665
EarlyStopping counter: 1 out of 40
EarlyStopping counter: 1 out of 40
EarlyStopping counter: 1 out of 40
EarlyStopping counter: 1 out of 40
E

In [8]:
import joblib
import torch
import numpy as np
import pandas as pd
from sklearn.metrics import r2_score, mean_absolute_error
x_scaler = joblib.load('torch_1/min_x_scaler.pkl')

C:\Users\User\anaconda3\Lib\site-packages\sklearn\base.py:442: InconsistentVersionWarning: Trying to unpickle estimator MinMaxScaler from version 1.8.0 when using version 1.7.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [9]:
early_stopping.load_best_model(model)
model.eval()

x_true = []
x_pred = []

with torch.no_grad():
    for x_t,x2_t, c_t in test_loader:
        x_t,x2_t, c_t = x_t.to(device),x2_t.to(device), c_t.to(device)
        bce_logit, x_hat,z_mu,z_logvar,z2_mu,z2_logvar = model(x_t,x2_t,c_t)
        x_true.append(x2_t.cpu().numpy())
        x_pred.append(x_hat.cpu().numpy())
all_x_true = np.vstack(x_true)
all_x_hat = np.vstack(x_pred)

Restored best model with loss: 0.785576


In [10]:
x_true_raw = x_scaler.inverse_transform(all_x_true)
x_hat_raw = x_scaler.inverse_transform(all_x_hat)
x_hat_raw = np.clip(x_hat_raw, 0,None)

In [11]:
from sklearn.metrics import r2_score, mean_squared_error
x_true = x_true_raw.flatten()
x_pred = x_hat_raw.flatten()
rmse = np.sqrt(mean_squared_error(x_true,x_pred))
r2 = r2_score(x_true,x_pred)
history = {'RMSE':round(float(rmse),4),'R2':round(float(r2),4)}
history

{'RMSE': 0.6762, 'R2': 0.8455}